In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, cross_val_score
# from tpot.config.classifier_nn import classifier_config_nn

from sklearn.pipeline import make_pipeline
# from tpot.config import classifier_config_dict_light
from tpot.config import classifier_config_dict
from sklearn.neighbors import KNeighborsClassifier


import pandas as pd
import numpy as np
import os
import glob

import time

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
pd.options.mode.chained_assignment = None

C:\Users\monicai\AppData\Local\Continuum\anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\monicai\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# personal_config = classifier_config_dict_light
personal_config = classifier_config_dict
personal_config['tpot.builtins.SimpleAutoencoder'] = {
    'encoding_dim': [10, 50, 100, 500],
    'activation': ['relu'],
    'optimizer': ['adadelta'],
    'loss':['binary_crossentropy'],
    'epochs':[10],
    'batch_size':[10, 15]
}

# personal_config['tpot.builtins.SimpleAutoencoder'] = {
#     'regularizer' :['regularizers.l1(10e-5)', 'regularizers.l2(10e-5)', 'regularizers.l1_l2(10e-5, 10e-5)'],
#     'encoding_dim': [40],
#     'activation': ['relu'],
#     'optimizer': ['adadelta', "SGD"],
#     'loss':['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
#        'squared_hinge', 'hinge', 'logcosh', 'binary_crossentropy', 'kullback_leibler_divergence', 'cosine_proximity', 'poisson'],
#     'epochs':[100],
#     'batch_size':[200]
# }

In [3]:
#new data
new_data = pd.read_csv("processed_aml_data.csv")
X = new_data.iloc[:, 1:5051]
y = new_data.iloc[:, 5051]

In [4]:
#convert X and y to numpy
X = X.values
y = y.values



In [5]:
#split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X, y, train_size = 0.8, test_size = 0.20, random_state = 42)
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_val = x_val.reshape((len(x_val)), np.prod(x_val.shape[1:]))

In [6]:
x_train.shape

(42, 5050)

In [ ]:
y_train.shape

In [7]:
from tpot.builtins import SimpleAutoencoder

In [8]:
sae = SimpleAutoencoder(encoding_dim=10, 
                        activation='relu', 
                        optimizer='adadelta', 
                        loss='binary_crossentropy', 
                        epochs=50, batch_size=10)

In [9]:
sae.fit(x_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


SimpleAutoencoder(activation='relu', batch_size=10, encoding_dim=10,
         epochs=50, loss='binary_crossentropy', optimizer='adadelta',
         random_state=42)

In [8]:
tpot = TPOTClassifier(generations=10, config_dict=personal_config,
                        population_size=10, verbosity=3,
                        template = 'SimpleAutoencoder-RandomForestClassifier')

In [ ]:
time = time.time()
tpot.fit(x_train, y_train)
end = time.time()
print(end - start)

31 operators have been imported by TPOT.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


Generation 1 - Current Pareto front scores:
-2	0.676984126984127	RandomForestClassifier(SimpleAutoencoder(input_matrix, SimpleAutoencoder__activation=relu, SimpleAutoencoder__batch_size=10, SimpleAutoencoder__encoding_dim=100, SimpleAutoencoder__epochs=10, SimpleAutoencoder__loss=binary_crossentropy, SimpleAutoencoder__optimizer=adadelta), RandomForestClassifier__bootstrap=True, RandomForestClassifier__criterion=gini, RandomForestClassifier__max_features=0.4, RandomForestClassifier__min_samples_leaf=6, RandomForestClassifier__min_samples_split=19, RandomForestClassifier__n_estimators=100)

Generation 2 - Current Pareto front scores:
-2	0.676984126984127	RandomForestClassifier(SimpleAutoencoder(input_matrix, SimpleAutoencoder__activation=relu, SimpleAutoencoder__batch_size=10, SimpleAutoencoder__encoding_dim=100, SimpleAutoencoder__epochs=10, SimpleAutoencoder__loss=binary_crossentropy, SimpleAutoencoder__optimizer=adadelta), RandomForestClassifier__bootstrap=True, RandomForestClassifie

In [12]:
tpot.fitted_pipeline_

Pipeline(memory=None,
     steps=[('simpleautoencoder', SimpleAutoencoder(activation='relu', batch_size=10, encoding_dim=10,
         epochs=10, loss='binary_crossentropy', optimizer='adadelta',
         random_state=42)), ('randomforestclassifier', RandomForestClassifier(bootstrap=False, class_weight=None,
            criter...ors=100, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False))])

In [13]:
tpot.score(x_train, y_train)

0.7857142857142857

In [14]:
tpot.score(x_val, y_val)

0.7272727272727273